In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
import pandas as pd
import math
from datetime import datetime , timedelta
from googletrans import Translator

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/colab_datasets/twitterscraping/datasetprime.csv', index_col=None)
df2 = pd.read_csv('/content/drive/MyDrive/colab_datasets/twitterscraping/datasetsdatasetprime.csv', index_col=None)
df3 = pd.read_csv('/content/drive/MyDrive/colab_datasets/twitterscraping/datasetprimer.csv', index_col=None)
df4 = pd.read_csv('/content/drive/MyDrive/colab_datasets/twitterscraping/datasetpriming.csv', index_col=None)
dfs = [df1,df2,df3,df4]
df = pd.concat(dfs).drop_duplicates()

In [ ]:
testdf = df.copy().drop(columns=['Retweet']).drop_duplicates()

In [ ]:
testdf = testdf.drop([testdf[testdf['Content'] == 'Content'].index[0]])

In [ ]:
def pipelinenumberoftweets(df):
  for i in range(df['NumberofTweets'].shape[0]):
    if (df['NumberofTweets'].iloc[i].find("Tweet") != -1):
      itm = df['NumberofTweets'].iloc[i][:-7]
      if itm == 'Numbero':
        df.drop([i])
      if ',' in itm:
        itm = itm.split(',')[0] + itm.split(',')[1]
      if 'K' in itm:
        try:
          itm = itm[:-1].split('.')[0] + itm[:-1].split('.')[1] + '00'
        except:
          itm = itm[:-1] + '000'
      if 'M' in itm:
        try:
          itm = itm[:-1].split('.')[0] + itm[:-1].split('.')[1] + '00000'
        except:
          itm = itm[:-1] + '000000'
      if itm == '':
        df['NumberofTweets'].iloc[i] = "0"
      else:
        df['NumberofTweets'].iloc[i] = itm
  df['NumberofTweets'] = pd.to_numeric(df['NumberofTweets'])

In [ ]:
def pipelinenumberoffollow(df):
  for j in ['NumberofFollowing','NumberofFollowers']:
    for i in range(df[j].shape[0]):
      itm = df[j].iloc[i]
      if itm[:14] == 'NumberofFollow':
        df.drop([i])
      if ',' in itm:
        itm = itm.split(',')[0] + itm.split(',')[1]
      if 'K' in itm:
        try:
          itm = itm[:-1].split('.')[0] + itm[:-1].split('.')[1] + '00'
        except:
          itm = itm[:-1] + '000'
      if 'M' in itm:
        try:
          itm = itm[:-1].split('.')[0] + itm[:-1].split('.')[1] + '00000'
        except:
          itm = itm[:-1] + '000000'
      if itm == '':
        df[j].iloc[i] = "0"
      else:
        df[j].iloc[i] = itm
    df[j] = pd.to_numeric(df[j])

In [ ]:
def pipelinenumberofys(df):
  for j in ['NumberOfComments','NumberOfRetweets','NumberOfLikes']:
    print(j)
    for i in range(df[j].shape[0]):
      itm = df[j].iloc[i]
      if (isinstance(itm, str)) :
        if ('\n' in itm):
          itm = itm.split('\n')[0]
        if (itm[:8] == 'NumberOf'):
          df.drop([i])
        if ',' in itm:
          itm = itm.split(',')[0] + itm.split(',')[1]
        if 'K' in itm:
          try:
            itm = itm[:-1].split('.')[0] + itm[:-1].split('.')[1] + '00'
          except:
            itm = itm[:-1] + '000'
        if 'M' in itm:
          try:
            itm = itm[:-1].split('.')[0] + itm[:-1].split('.')[1] + '00000'
          except:
            itm = itm[:-1] + '000000'
        if itm == '':
          df[j].iloc[i] = "0"
        else:
          df[j].iloc[i] = itm
      elif isinstance(itm, float):
        df[j].iloc[i] = "0"
    df[j] = pd.to_numeric(df[j])

In [ ]:
def guesslang(df):
  translator = Translator()
  df['language'] = 'unknown'
  for i in range(df['Content'].shape[0]):
    if isinstance(df['Content'].iloc[i],str):
      try:
        df['language'].iloc[i] = translator.detect(df['Content'].iloc[i]).lang
      except:
        pass

In [ ]:
def countnumberoftags(df):
  df['numberOfHashtags'] = 0
  df['numberOfTags'] = 0
  df['numberOfURLs'] = 0
  for i in range(df['Content'].shape[0]):
    if isinstance(df['Content'].iloc[i],str):
      df['numberOfHashtags'].iloc[i] = df['Content'].iloc[i].count('#')
      df['numberOfTags'].iloc[i] = df['Content'].iloc[i].count('@')
      df['numberOfURLs'].iloc[i] = df['Content'].iloc[i].count('http')

In [ ]:
pipelinenumberoftweets(testdf)
pipelinenumberoffollow(testdf)
pipelinenumberofys(testdf)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


NumberOfComments
NumberOfRetweets
NumberOfLikes


In [ ]:
testdf = testdf.drop(columns=['NumberOfCommentslast','NumberOfRetweetslast','NumberOfLikeslast'])

In [ ]:
testdf['timeoftweet'] = testdf.timeoftweet.apply(lambda x: x[:10] + ' ' + x[11:19])
testdf['timeoftweet'] = pd.to_datetime(testdf['timeoftweet'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
testdf['timeofscraping'] = testdf.timeofscraping.apply(lambda x: x[:10] + ' ' + x[11:19])
testdf['timeofscraping'] = pd.to_datetime(testdf['timeofscraping'], format='%Y-%m-%d %H:%M:%S') + timedelta(hours=-1)

In [ ]:
testdf['tweetage'] = 1
for i in range(testdf.shape[0]):
  if (pd.Timedelta("1 hours") < testdf['timeofscraping'].iloc[i] - testdf['timeoftweet'].iloc[i] <= pd.Timedelta("12 hours")):
    testdf["tweetage"].iloc[i] = 2
  if (pd.Timedelta("12 hours") < testdf['timeofscraping'].iloc[i] - testdf['timeoftweet'].iloc[i] <= pd.Timedelta("1 days")):
    testdf["tweetage"].iloc[i] = 3
  if (testdf['timeofscraping'].iloc[i] - testdf['timeoftweet'].iloc[i] > pd.Timedelta("1 days")):
    testdf["tweetage"].iloc[i] = 4

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
testdf = testdf.drop(columns=['timeoftweet','timeofscraping'])

In [ ]:
countnumberoftags(testdf)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
testdf = testdf.drop_duplicates()

In [ ]:
guesslang(testdf)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
#testdf['NumberofComments'] = testdf['NumberOfComments']
#testdf['NumberofRetweets'] = testdf['NumberOfRetweets']
#testdf['NumberofLikes'] = testdf['NumberOfLikes']
#testdf = testdf.drop(columns=['NumberOfRetweets','NumberOfComments','NumberOfLikes'])

In [ ]:
testdf.to_csv('/content/drive/MyDrive/colab_datasets/twitterscraping/alldatacleaned.csv',index=False)